In [58]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Carregar os dados
df = pd.read_excel('brSerieA.xlsx')

# Criar o app
app = Dash(__name__)

# Anos disponíveis
anos_disponiveis = sorted(df['ano_campeonato'].unique())

# Função para calcular o time que mais pontuou
def time_mais_pontuou(df, ano):
    df_ano = df[df['ano_campeonato'] == ano]
    pontos_mandante = df_ano.groupby('time_mandante')['gols_mandante'].sum()
    pontos_visitante = df_ano.groupby('time_visitante')['gols_visitante'].sum()
    pontos_total = pontos_mandante.add(pontos_visitante, fill_value=0)
    return pontos_total

# Função para calcular o time que mais venceu
def time_mais_vitorias(df, ano):
    df_ano = df[df['ano_campeonato'] == ano]
    vitorias_mandante = df_ano[df_ano['gols_mandante'] > df_ano['gols_visitante']].groupby('time_mandante').size()
    vitorias_visitante = df_ano[df_ano['gols_visitante'] > df_ano['gols_mandante']].groupby('time_visitante').size()
    vitorias_total = vitorias_mandante.add(vitorias_visitante, fill_value=0)
    return vitorias_total

# Função para verificar o melhor mandante
def melhor_mandante(df, ano):
    df_ano = df[df['ano_campeonato'] == ano]
    vitorias_mandante = df_ano[df_ano['gols_mandante'] > df_ano['gols_visitante']].groupby('time_mandante').size()
    return vitorias_mandante

# Layout da aplicação
app.layout = html.Div(children=[
    html.H1(children='Campeonato Brasileiro Série A'),
    
    # Dropdown para selecionar o ano
    html.Label('Selecione o ano:'),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': str(ano), 'value': ano} for ano in anos_disponiveis],   
        value=anos_disponiveis[0],  # Define o primeiro ano disponível como padrão
        clearable=False
    ),
    
    # Dropdown para selecionar a questão
    html.Label('Selecione a questão:'),
    dcc.Dropdown(
        id='questao-dropdown',
        options=[
            {'label': 'Time que mais pontuou', 'value': 'mais_pontuou'},
            {'label': 'Time que mais venceu', 'value': 'mais_venceu'},
            {'label': 'Melhor mandante', 'value': 'melhor mandante'},
        ],
        value='mais_pontuou',  # Define a primeira questão como padrão
        clearable=False
    ),
    
    # Gráfico de barras para mostrar o resultado
    dcc.Graph(id='grafico-resultados'),
])

# Callback para atualizar o gráfico baseado na seleção
@app.callback(
    Output('grafico-resultados', 'figure'),
    [Input('ano-dropdown', 'value'),
     Input('questao-dropdown', 'value')]
)
def exibir_grafico(ano, questao):
    if questao == 'mais_pontuou':
        pontos_total = time_mais_pontuou(df, ano)
        # Criação do gráfico de barras para os pontos totais
        fig = px.bar(
            pontos_total,
            x=pontos_total.index,
            y=pontos_total.values,
            labels={'x': 'Time', 'y': 'Total de Pontos'},
            title=f'Pontos por time em {ano}'
        )
    elif questao == 'mais_venceu':
        vitorias_total = time_mais_vitorias(df, ano)
        # Criação do gráfico de barras para as vitórias totais
        fig = px.bar(
            vitorias_total,
            x=vitorias_total.index,
            y=vitorias_total.values,            
            labels={'x': 'Times', 'y': 'Total de Vitórias'},
            title=f'Vitórias por time em {ano}'
        )
        
    elif questao == 'melhor mandante':
        vitorias_mandante_total = melhor_mandante(df, ano)
        # Criação do gráfico de barras para as vitórias totais
        fig = px.bar(
            vitorias_mandante_total,
            x=vitorias_mandante_total.index,
            y=vitorias_mandante_total.values,            
            labels={'x': 'Times', 'y': 'Total de vitórias'},
            title=f'Vitórias por time em {ano}'
        )
    
    # Retornar a figura
    return fig

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
def calcular_vitorias(df, ano=None):
    if ano:
        df = df[df['ano_campeonato'] == ano].copy()  # Fazendo uma cópia explícita para evitar o aviso de erro 'SettingWithCopyWarning'
    df['vencedor'] = df.apply(lambda row: row['time_mandante'] if row['gols_mandante'] > row['gols_visitante'] 
                              else row['time_visitante'] if row['gols_visitante'] > row['gols_mandante'] 
                              else 'Empate', axis=1)
    df_vitorias = df[df['vencedor'] != 'Empate'].groupby('vencedor').size().reset_index(name='vitorias')
    df_vitorias['pontos'] = df_vitorias['vitorias'] * 3
    df_vitorias['pontosTotais'] = df_vitorias['pontos'] + df[df['vencedor'] == 'Empate'].shape[0]
    df_pontuacao = 


    return df_vitorias
